In [2]:
import nltk
import string
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import snowball
from nltk.corpus import stopwords
import keras
from keras import *
from keras import layers
from keras.preprocessing import text
from keras.preprocessing import sequence
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import PyPDF2
from glob import glob, glob1
import numpy as np
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\doonl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\doonl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
path = './txtFile/'

data = pd.DataFrame(columns=['id', 'abstract', 'keywords'])
for i in glob1('./txtFile/', '*'):
    with open(path + i, 'r', encoding='utf8') as f:
        doc_id = i.split('.')[-2]
        readingAbstract = False
        readingKeywords = False
        abstract_sent = []
        keyword_list = []
        keyword_clean = []
        for j in f:
            line = j.strip()
            if line == 'Keywords:' and readingKeywords is False:
                readingKeywords = True
                readingAbstract = False
                continue
            if line == 'Abstract' and readingAbstract is False:
                readingAbstract = True
                continue
            if readingAbstract is True:
                abstract_sent.append(line)
            if readingKeywords is True:
                keyword_list.append(line)
        abstract_para = ' '.join(abstract_sent)
        keyword_split = keyword_list[0].split(',')
        # print(abstract_para)
        # print(keyword_split)
        for x in keyword_split:
            keyword_lower = x.lower()
            keyword_no_space = keyword_lower.replace(' ', '')
            keyword_underscore = keyword_no_space.replace('_', ' ')
            keyword_clean.append(keyword_underscore)
        # print(keyword_clean)

        data = data.append({"id": doc_id, "abstract": abstract_para, "keywords": keyword_clean}, ignore_index=True)
    f.close()

display(data)


C:\Users\doonl\AppData\Local\Temp\ipykernel_25912\2526147678.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"id": doc_id, "abstract": abstract_para, "keywords": keyword_clean}, ignore_index=True)
C:\Users\doonl\AppData\Local\Temp\ipykernel_25912\2526147678.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"id": doc_id, "abstract": abstract_para, "keywords": keyword_clean}, ignore_index=True)
C:\Users\doonl\AppData\Local\Temp\ipykernel_25912\2526147678.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"id": doc_id, "abstract": abstract_para, "keywords": keyword_clean}, ignore_index=True)
C:\Users\doonl\AppData\Local\Temp\ipykernel_25912\2526147678.py:36:

,id,abstract,keywords
0,1,"In the logistic management, the cost reduction...","[artificial intelligence, combinatorial optimi..."
1,10,Word segmentation is an important pre-processi...,"[attention, deep learning, thai language, word..."
2,11,Loss estimation is considerably significant fo...,"[network resource planning, traffic model, que..."
3,12,This thesis presents a computational study on...,"[audio-synchronization, facial expression, fac..."
4,13,The financial problem is a challenging task fo...,"[deep learning, distributional reinforcement l..."
5,14,Since a classical computer has a limitation to...,"[circuit depth, computing wall-time, quantum r..."
6,15,Hyperparameter Tuning chooses optimal hyperpar...,"[abc, anova, classification, grid search, hype..."
7,16,The honeybee is a social insect that communica...,"[convolutional neural networks, feature extrac..."
8,17,System logs contain the complete information o...,"[alice, anomaly detection, cern, convolutional..."
9,18,Handling an imbalanced class problem is a chal...,"[classification, imbalanced dataset, machine l..."


In [4]:
data.count()

id          18
abstract    18
keywords    18
dtype: int64

In [5]:
data_explode = data.explode('keywords').reset_index(drop=True)

In [6]:
data_explode

,id,abstract,keywords
0,1,"In the logistic management, the cost reduction...",artificial intelligence
1,1,"In the logistic management, the cost reduction...",combinatorial optimization
2,1,"In the logistic management, the cost reduction...",vehicle routing problem
3,1,"In the logistic management, the cost reduction...",hard time windows
4,1,"In the logistic management, the cost reduction...",meta heuristic
...,...,...,...
78,9,Understanding and interpretation of legislativ...,coreference resolution
79,9,Understanding and interpretation of legislativ...,legal processing
80,9,Understanding and interpretation of legislativ...,natural language processing
81,9,Understanding and interpretation of legislativ...,semantic meaning representations


In [7]:
data_u = data_explode['keywords'].unique()
data_u

array(['artificial intelligence', 'combinatorial optimization',
       'vehicle routing problem', 'hard time windows', 'meta heuristic',
       'genetic algorithm', 'attention', 'deep learning', 'thai language',
       'word segmentation', 'network resource planning', 'traffic model',
       'queuing network', 'stochastic model', 'audio-synchronization',
       'facial expression', 'facial movement', 'speech production',
       'distributional reinforcement learning', 'portfolio management',
       'circuit depth', 'computing wall-time', 'quantum random walk',
       'quantum computer simulator', 'abc', 'anova', 'classification',
       'grid search', 'hyperparameter tuning', 'random search',
       'convolutional neural networks', 'feature extraction',
       'hissing signal', 'machine learning', 'monitoring beehive',
       'alice', 'anomaly detection', 'cern',
       'convolutional neural network', 'self-attention', 'transformer',
       'imbalanced dataset', 'oversampling', 'probab

In [8]:
data_explode['keywords'].duplicated().any

<bound method NDFrame._add_numeric_operations.<locals>.any of 0     False
1     False
2     False
3     False
4     False
      ...  
78    False
79    False
80     True
81    False
82    False
Name: keywords, Length: 83, dtype: bool>

In [9]:
data_explode['word_id'] = pd.factorize(data_explode.keywords)[0]
data_explode

,id,abstract,keywords,word_id
0,1,"In the logistic management, the cost reduction...",artificial intelligence,0
1,1,"In the logistic management, the cost reduction...",combinatorial optimization,1
2,1,"In the logistic management, the cost reduction...",vehicle routing problem,2
3,1,"In the logistic management, the cost reduction...",hard time windows,3
4,1,"In the logistic management, the cost reduction...",meta heuristic,4
...,...,...,...,...
78,9,Understanding and interpretation of legislativ...,coreference resolution,66
79,9,Understanding and interpretation of legislativ...,legal processing,67
80,9,Understanding and interpretation of legislativ...,natural language processing,52
81,9,Understanding and interpretation of legislativ...,semantic meaning representations,68


In [10]:
data_explode[data_explode['keywords'] == 'deep learning'].tail()

,id,abstract,keywords,word_id
7,10,Word segmentation is an important pre-processi...,deep learning,7
19,13,The financial problem is a challenging task fo...,deep learning,7
57,4,Due to the emerging of Long Short-Term Memory ...,deep learning,7
66,6,This thesis proposed a speech production acqui...,deep learning,7


In [11]:
data_explode.describe()

,word_id
count,83.000000
mean,33.337349
std,20.157476
min,0.000000
25%,15.500000
50%,33.000000
75%,51.500000
max,69.000000


In [12]:
x = np.array(data_explode['abstract'].values)
y = np.array(data_explode['word_id'].values)

In [13]:
len(y)

83

In [14]:
x

array(['In the logistic management, the cost reduction for delivering the  goods to customer s is crucial to save the budget of the company . Since decades ago, various Vehicle Routing Problems (VRPs ) have been emerged enormously to improve the productivity and to reduce the logistic cost of the industry . Among them, Vehicle Routing Problem with Time Windows  (VRPTW ) is one of the most fundamental VRP variants and one of the most applicable variants in the real-world case studies. In this study , VRPTW with  hard time windows is solved  by developing a special Genetic Algorithm (GA), composed of a problem -specific crossover operator and seven different mutation operators. The proposed GA has better results with  the heuristic mutation among seven operators while exploring the new and better features in large search space. The results of the algorithm are tested on the popular Solomon benchmark 100 - customer datasets. The results show that the proposed GA is quite comparable  with 

In [15]:
x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.1, random_state=0)


In [16]:
len(x_train[0])
x_train

array(['Word segmentation is an important pre-processing step in natural language processing applications, particularly in languages with no demarcation indicators including such as Thai. Dictionary-based segmentation, while simple, does not consider the context of the sentence. This paper proposes an attention-based deep learning approach for Thai word segmentation. With an additional attention mechanism, the model can learn character correlations across the entire sentence without gradient vanishing or gradient explode problems and then tokenize them into word vectors. The goal of this research is to test two different types of attention mechanisms to determine the effectiveness of word tokenization. The visualization of attention for each attention mechanism is also included as an outcome.',
       'In recent studies, many NLP tasks could gain better performance by applying the word embedding as the representation of words. In this research , we propose Deep Word -Topic Latent Diric

In [17]:
t = text.Tokenizer()
t.fit_on_texts(x_train)
x_indx = t.texts_to_sequences(x_train)
x_indx_test = t.texts_to_sequences(x_test)

In [18]:
len(x_indx)
len(x_indx_test)

9

In [19]:
maxlen = len(max(x_indx, key=len))


In [20]:
len(t.word_index)

1136

In [21]:
for i, x in enumerate(x_indx):
    if len(x) < maxlen:
        x_indx[i] = np.hstack((x, np.zeros(maxlen-len(x))))
x_train_index = np.array(x_indx)

In [22]:
x_train_index.shape

(74, 278)

In [23]:
x_train_index[30]

array([  1.,  62.,   3.,  97.,   5.,   6.,  41., 544.,   8.,   1., 545.,
         7., 546.,  91.,   1., 371.,   3.,  18., 291.,  68., 547.,  13.,
        97.,   5.,  12.,  44., 222., 150., 548.,  34.,   2., 195.,  31.,
         1.,  12., 549.,  14.,  44.,   3.,   1.,  97.,   5.,   6.,  41.,
        28.,  50.,  95.,  18.,  74., 186.,   3.,  48.,  11., 111., 196.,
         6., 292.,  30.,   8.,  97.,   9., 550.,   1.,  44., 551.,  45.,
        39.,  97.,  31.,   6.,  41.,   1.,  12.,  19., 293.,  13.,  14.,
        68., 160.,  24.,   3.,   1., 552., 553.,   3., 554., 555.,   1.,
        30., 372.,  26., 556.,   2.,  17.,  44.,   4., 194., 557.,  13.,
         1.,  82.,   1.,  44.,  42., 332., 373., 558.,  44., 559., 560.,
         2., 561., 246.,   1.,  30., 171., 562.,  14., 563., 564., 294.,
        24.,  13.,   1.,  17.,  48., 565.,  13.,   1.,  17.,  44.,   1.,
        78., 223.,  14., 374.,   5.,   1.,  23., 247., 363.,   1.,  17.,
        48.,   9., 291.,  48.,  11., 566.,   1., 16

In [24]:
total_word = len(t.word_index) +1

In [25]:
t.word_index

{'the': 1,
 'and': 2,
 'of': 3,
 'to': 4,
 'in': 5,
 'a': 6,
 'is': 7,
 'on': 8,
 'with': 9,
 'for': 10,
 'this': 11,
 'data': 12,
 'from': 13,
 'an': 14,
 'model': 15,
 'that': 16,
 'network': 17,
 'using': 18,
 'were': 19,
 'are': 20,
 'based': 21,
 'amr': 22,
 'performance': 23,
 'system': 24,
 'learning': 25,
 'text': 26,
 'was': 27,
 'can': 28,
 'by': 29,
 'study': 30,
 'as': 31,
 'coreference': 32,
 'meaning': 33,
 'time': 34,
 'applied': 35,
 'proposed': 36,
 'has': 37,
 'results': 38,
 'legal': 39,
 'word': 40,
 'dataset': 41,
 'method': 42,
 'models': 43,
 'analysis': 44,
 'thai': 45,
 'research': 46,
 'which': 47,
 'features': 48,
 'it': 49,
 'be': 50,
 'algorithm': 51,
 'been': 52,
 'processing': 53,
 'different': 54,
 'representation': 55,
 'machine': 56,
 'problem': 57,
 'we': 58,
 'our': 59,
 'used': 60,
 'new': 61,
 'number': 62,
 'problems': 63,
 'process': 64,
 'quantum': 65,
 'many': 66,
 'best': 67,
 'information': 68,
 'tuning': 69,
 'lstm': 70,
 'ga': 71,
 'techniq

In [26]:
total_word

1137

In [27]:
def perplexity(y_true, y_pred):
    cross_entropy = keras.backend.categorical_crossentropy(y_true, y_pred)
    perplexity = keras.backend.pow(2.0, cross_entropy)
    return perplexity


In [28]:
model = keras.Sequential()

model.add(keras.layers.Embedding(total_word, 100, input_length=maxlen, name='Embedding'))
model.add(keras.layers.LSTM(512, kernel_initializer='he_normal', dropout=0.3, return_sequences=True))
model.add(keras.layers.LSTM(256, kernel_initializer='he_normal', dropout=0.3))
model.add(keras.layers.Dense(70, activation='softmax'))
# model.add(keras.layers.Dense(70))

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam' ,metrics=['acc'])



In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Embedding (Embedding)       (None, 278, 100)          113700    
                                                                 
 lstm (LSTM)                 (None, 278, 512)          1255424   
                                                                 
 lstm_1 (LSTM)               (None, 256)               787456    
                                                                 
 dense (Dense)               (None, 70)                17990     
                                                                 
Total params: 2,174,570
Trainable params: 2,174,570
Non-trainable params: 0
_________________________________________________________________


In [30]:
y_train = np.array(y_train)

In [31]:
len(y_train)

74

In [32]:
his = model.fit(x_train_index, y_train, epochs=10, batch_size= 32)

Epoch 1/10
3/3 [==============================] - 28s 7s/step - loss: 4.3130 - acc: 0.0000e+00
Epoch 2/10
3/3 [==============================] - 30s 9s/step - loss: 4.2094 - acc: 0.0541
Epoch 3/10
3/3 [==============================] - 36s 9s/step - loss: 4.1565 - acc: 0.0541
Epoch 4/10
3/3 [==============================] - 29s 9s/step - loss: 4.1110 - acc: 0.0811
Epoch 5/10
3/3 [==============================] - 29s 9s/step - loss: 4.1086 - acc: 0.0946
Epoch 6/10
3/3 [==============================] - 29s 9s/step - loss: 4.0407 - acc: 0.0811
Epoch 7/10
3/3 [==============================] - 32s 9s/step - loss: 4.0171 - acc: 0.0541
Epoch 8/10
3/3 [==============================] - 29s 8s/step - loss: 4.1478 - acc: 0.0676
Epoch 9/10
3/3 [==============================] - 28s 9s/step - loss: 3.9877 - acc: 0.0541
Epoch 10/10
3/3 [==============================] - 28s 8s/step - loss: 3.9358 - acc: 0.0541
